# Registration
## Abstract
- This section describe registration algorithm for point cloud registration.

## Introduction


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Use Iterative Closest Point (ICP)

This subsection demonstrates registration with ICP for a object point cloud. The subsection tutorial use following packages:

In [11]:
import numpy as np

from tutlibs.registration import icp
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.transformation import rotation, translation
from tutlibs.transformation import TransformationMatrix as tm

The tutorial code is following:

In [12]:
# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = rotation(translation(target_xyz, np.array([0.0, 0.1, 0.0])), 'z', angle=30/180*np.pi)

# # keypoints estimation
source_v_xyz = voxel_grid_sampling(source_xyz, 0.01)
target_v_xyz = voxel_grid_sampling(target_xyz, 0.01)

# use ICP
trans_mat = icp(source_v_xyz, target_v_xyz, 20)
estimated_source_xyz = tm.transformation_Nx3_with_4x4(source_xyz, trans_mat)

# visualize results
source_colors = np.tile([[1, 0, 0]], (source_xyz.shape[0], 1))
estimated_source_colors = np.tile([[0, 1, 0]], (estimated_source_xyz.shape[0], 1))
target_colors = np.tile([[1, 1, 0]], (target_xyz.shape[0], 1))

vis_xyz = np.concatenate([estimated_source_xyz, target_xyz, source_xyz])
vis_color = np.concatenate([estimated_source_colors, target_colors, source_colors])
visualizer.k3d(vis_xyz, colors=vis_color)
# io.write('outputs/r1.ply', vis_xyz, vis_color)


Output()

## Use ICP and RANSAC

This subsection demonstrates object regustration with ICP and point correspondence RANSAC. The subsection tutorial use following packages:

In [13]:
import numpy as np

from tutlibs.registration import icp_ransac
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.transformation import rotation, translation
from tutlibs.transformation import TransformationMatrix as tm

The tutorial code is following:

In [16]:
# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = rotation(translation(target_xyz, np.array([0.0, 0.1, 0.0])), 'z', angle=30/180*np.pi)

# # keypoints estimation
source_v_xyz = voxel_grid_sampling(source_xyz, 0.01)
target_v_xyz = voxel_grid_sampling(target_xyz, 0.01)

# use ICP
trans_mat = icp_ransac(source_v_xyz, target_v_xyz, 20)
estimated_source_xyz = tm.transformation_Nx3_with_4x4(source_xyz, trans_mat)

# visualize results
source_colors = np.tile([[1, 0, 0]], (source_xyz.shape[0], 1))
estimated_source_colors = np.tile([[0, 1, 0]], (estimated_source_xyz.shape[0], 1))
target_colors = np.tile([[1, 1, 0]], (target_xyz.shape[0], 1))

vis_xyz = np.concatenate([estimated_source_xyz, target_xyz, source_xyz])
vis_color = np.concatenate([estimated_source_colors, target_colors, source_colors])
visualizer.k3d(vis_xyz, colors=vis_color)
# io.write('outputs/r1.ply', vis_xyz, vis_color)


Output()

## Use of Open3D ICP

This subsection demonstrates registration with Open3D ICP for a object point cloud. The subsection tutorial use following packages:

In [4]:
import numpy as np
import open3d as o3d

from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.transformation import rotation, translation
from tutlibs.transformation import TransformationMatrix as tm

The tutorial code is following:

In [14]:
def icp_o3d(source_xyz, target_xyz, threshold=0.1):
    source = o3d.geometry.PointCloud()
    target = o3d.geometry.PointCloud()

    source.points = o3d.utility.Vector3dVector(source_xyz)
    target.points = o3d.utility.Vector3dVector(target_xyz)

    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))

    transformation = reg_p2p.transformation

    return transformation

# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = rotation(translation(target_xyz, np.array([0.0, 0.1, 0.0])), 'z', angle=30/180*np.pi)

# # keypoints estimation
source_v_xyz = voxel_grid_sampling(source_xyz, 0.01)
target_v_xyz = voxel_grid_sampling(target_xyz, 0.01)

# use ICP
trans_mat = icp_o3d(source_v_xyz, target_v_xyz)
estimated_source_xyz = tm.transformation_Nx3_with_4x4(source_xyz, trans_mat)

# visualize results
source_colors = np.tile([[1, 0, 0]], (source_xyz.shape[0], 1))
estimated_source_colors = np.tile([[0, 1, 0]], (estimated_source_xyz.shape[0], 1))
target_colors = np.tile([[1, 1, 0]], (target_xyz.shape[0], 1))

vis_xyz = np.concatenate([estimated_source_xyz, target_xyz, source_xyz])
vis_color = np.concatenate([estimated_source_colors, target_colors, source_colors])
visualizer.k3d(vis_xyz, colors=vis_color)
# io.write('outputs/r1.ply', vis_xyz, vis_color)


Output()